In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from scipy.stats import kstest
from sklearn.metrics import precision_score
import scipy
from statsmodels.stats.diagnostic import kstest_normal
from scipy.stats import norm

In [ ]:
# load everything
NonS = np.loadtxt("drive/MyDrive/EEG_recordings/NonS.txt")
PreS = np.loadtxt("drive/MyDrive/EEG_recordings/PreS.txt")
Seizure = np.loadtxt("drive/MyDrive/EEG_recordings/Seizure.txt")
Test2 = np.loadtxt("drive/MyDrive/EEG_recordings/ProjectTestDataSet2.txt")

EEG = np.hstack((NonS, PreS, Seizure))
print(EEG.shape)
print(PreS.shape)
print(NonS.shape)
print(Seizure.shape)
print(Test2.shape)

Test1N = np.loadtxt("drive/MyDrive/EEG_recordings/TESTDATASET1.txt", max_rows=12)
Test1P = np.loadtxt("drive/MyDrive/EEG_recordings/TESTDATASET1.txt", skiprows=12, max_rows=12)
Test1S = np.loadtxt("drive/MyDrive/EEG_recordings/TESTDATASET1.txt", skiprows=24, max_rows=12)
EEG2 = np.hstack((Test1N, Test1P, Test1S))
print(EEG2.shape)
print([Test1N.shape[1], Test1N.shape[1] + Test1P.shape[1]], EEG2.shape[1])

NewTest1 = np.loadtxt("drive/MyDrive/EEG_recordings/TestFile1.txt")
NewTest2 = np.loadtxt("drive/MyDrive/EEG_recordings/TestFile2.txt")
NewTest3 = np.loadtxt("drive/MyDrive/EEG_recordings/TestFile3.txt")

Earth2 = np.loadtxt("drive/MyDrive/EEG_recordings/2016.008.01.47.28.IL22.HHZ.txt")
Earth3 = np.loadtxt("drive/MyDrive/EEG_recordings/2016.008.01.47.28.IL07.HHZ.txt")

print("shapes")
print(NewTest1.shape, NewTest2.shape, NewTest3.shape, Earth2.shape, Earth3.shape)

(12, 170752)
(12, 76800)
(12, 76800)
(12, 17152)
(9819,)
(12, 165760)
[76800, 153600] 165760
shapes
(12, 165760) (12, 38312) (12, 32040) (8782,) (8782,)


In [ ]:
#Baseline algorithm class
class BaselineCPD:

    #Intializing params
    def __init__(self, n0, M, K):

        self.n0 = n0
        self.M = M
        self.K = K
        self.AR_params = {}
        self.rejection_count = {}
        self.f0 = {}

    #Function to fit AR model
    def fit_model(self, data, max_lags=10):

        values = []

        for p in range(1, max_lags + 1):

            ar_model = AutoReg(data, lags=p, old_names=False).fit()
            values.append(ar_model.bic)

        best_order = np.argmin(values) + 1

        return best_order

    #Fitting model after warmup
    def warmup(self, data, s):

        T = s * self.n0
        for c in range(data.shape[0]):

            channel_data = data[c][:T]
            optimal_lag = self.fit_model(channel_data)
            ar_model = AutoReg(channel_data, lags=optimal_lag, old_names=False).fit()

            i = ar_model.resid
            mean_e = np.mean(i)
            std_e = np.std(i)

            self.AR_params[c] = {"model": ar_model, "lag": optimal_lag}
            self.f0[c] = norm(mean_e, std_e)
            self.rejection_count[c] = 0


    #Predict the next window using the AR model and perform hypothesis testing
    def predict_and_test(self, data, channel, window_size):

        ar_model = self.AR_params[channel]["model"]
        pred = ar_model.predict(start=len(data), end=len(data) + window_size - 1)
        e = data[channel][-window_size:] - pred

        return e, self.f0[channel].cdf(e)

    #Compute the cumulative divergence between two distributions
    def cumulative_divergence(self, e, f1, f0):

        divergence = np.sum(np.log(f1.pdf(e) / f0.pdf(e)))
        return divergence

    #Algorithm to detect CP
    def detect_change_point(self, data, s):

        cpd_detected = False

        for i in range(self.n0 * s, data.shape[1], s):

            for c in range(data.shape[0]):

                e, cdf_values = self.predict_and_test(data, c, s)
                #Using 5-sigma rile
                rejected = np.sum(cdf_values) > 125

                if rejected:
                    cpd_detected = True
                    break
                else:
                    self.rejection_count[c] = 0

            if cpd_detected:
                break

        return cpd_detected

In [ ]:
def detect_change_points(data, baseline_cpd, s):
    change_points = []

    for i in range(baseline_cpd.n0 * s, len(data[0]), 64 * s):
        #Detect if there's a change point
        cpd_detected = baseline_cpd.detect_change_point(data[:,:i], s)
        if cpd_detected:
            #Record the exact position of the change point
            change_points.append(i)

    return change_points

#Compute Average Run Length (ARL) and Expected Detection Delay (EDD) as metrics
def compute_arl_edd(detection, ground_truth):
    detection_early = []
    detection_delays = []

    for det_timestamp in detection:
        closest_gt = min(ground_truth, key=lambda x: abs(x - det_timestamp))
        if det_timestamp < closest_gt:
            detection_early.append(closest_gt - det_timestamp)

    for gt_timestamp in ground_truth:
        closest_det = min(detection, key=lambda x: abs(x - gt_timestamp))
        if closest_det > gt_timestamp:
            detection_delays.append(closest_det - gt_timestamp)

    arl = np.mean(detection_early) if detection_early else 0
    edd = np.mean(detection_delays) if detection_delays else 0

    return arl, edd

Initial Test Data (EEG)




In [ ]:
#Parameters: n0 - warmup period length, n1 - window size length, M - rejection threshold, K - cumulative divergence threshold
n0 = 60
n1 = 64
M = 3
K = 10
s = 128

baseline_cpd = BaselineCPD(n0, M, K)

#EEG data and ground truth change points
gt_bkps = [76800, 76800 * 2, 170752]

baseline_cpd.warmup(EEG, s)

#Prediction Phase: detect change points
predicted_bkps = detect_change_points(EEG, baseline_cpd, s)

true_positive = len(set(predicted_bkps).intersection(gt_bkps))
precision = true_positive / len(predicted_bkps) if predicted_bkps else 0

arl, edd = compute_arl_edd(predicted_bkps[:-1], gt_bkps[:-1])

print(f"Change points prediction: {predicted_bkps}")
print(f"Precision: {precision}")
print(f"Average run length (ARL): {arl}")
print(f"Expected detection delay (EDD): {edd}")

Change points prediction: [65024, 155136]
Precision: 0.0
Average run length (ARL): 11776.0
Expected detection delay (EDD): 0


In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(EEG[0], label='EEG Data')
for gt in gt_bkps:
    plt.axvline(x=gt, color='green', linestyle='--', label='Ground Truth CP')
plt.xlabel('Time')
plt.ylabel('EEG Signal')
plt.title('Ground Truth Change Points')
plt.legend()
plt.show()


# Visualize EEG data with detected and ground truth change points (Plotting first channel)
plt.figure(figsize=(15, 5))
plt.plot(EEG[0], label='EEG Data')
for cp in predicted_bkps:
    plt.axvline(x=cp, color='red', linestyle='--', label='Detected CP')
plt.xlabel('Time')
plt.ylabel('EEG Signal')
plt.title('DetectedChange Points')
plt.legend()
plt.show()

Test EEG Data

In [ ]:
#Parameters: n0 - warmup period length, n1 - window size length, M - rejection threshold, K - cumulative divergence threshold
n0 = 100
n1 = 64
M = 5
K = 10
s = 512

baseline_cpd = BaselineCPD(n0, M, K)


baseline_cpd.warmup(EEG2, s)

#Prediction Phase: detect change points
predicted_bkps = detect_change_points(EEG2, baseline_cpd, s)


print(f"Change points prediction: {predicted_bkps}")


Change points prediction: [83968, 116736, 149504]


In [ ]:
#Parameters: n0 - warmup period length, n1 - window size length, M - rejection threshold, K - cumulative divergence threshold
n0 = 80
n1 = 64
M = 3
K = 10
s = 512

baseline_cpd = BaselineCPD(n0, M, K)


baseline_cpd.warmup(NewTest1, s)

#Prediction Phase: detect change points
predicted_bkps = detect_change_points(NewTest1, baseline_cpd, s)


print(f"Change points prediction: {predicted_bkps}")


Change points prediction: [73728, 106496, 139264]


In [ ]:
#Parameters: n0 - warmup period length, n1 - window size length, M - rejection threshold, K - cumulative divergence threshold
n0 = 80
n1 = 64
M = 3
K = 10
s = 128

baseline_cpd = BaselineCPD(n0, M, K)


baseline_cpd.warmup(NewTest2, s)

#Prediction Phase: detect change points
predicted_bkps = detect_change_points(NewTest2, baseline_cpd, s)


print(f"Change points prediction: {predicted_bkps}")

Change points prediction: [18432, 26624]


In [ ]:
#Parameters: n0 - warmup period length, n1 - window size length, M - rejection threshold, K - cumulative divergence threshold
n0 = 10
n1 = 64
M = 3
K = 10
s = 64

baseline_cpd = BaselineCPD(n0, M, K)


baseline_cpd.warmup(NewTest3, s)

#Prediction Phase: detect change points
predicted_bkps = detect_change_points(NewTest3, baseline_cpd, s)


print(f"Change points prediction: {predicted_bkps}")

Change points prediction: []


Adjusted CPD Implementation for seismic data

In [ ]:
class BaselineCPDEarth:

    #Intializing params
    def __init__(self, n0, M, K):

        self.n0 = n0
        self.M = M
        self.K = K
        self.AR_params = {}
        self.rejection_count = 0
        self.f0 = {}

    #Function to fit AR model
    def fit_model(self, data, max_lags=10):

        values = []

        for p in range(1, max_lags + 1):

            ar_model = AutoReg(data, lags=p, old_names=False).fit()
            values.append(ar_model.bic)

        best_order = np.argmin(values) + 1

        return best_order


    #Fitting model after warmup
    def warmup(self, data, s):

        T = s * self.n0
        warmup_data = data[:T]

        optimal_lag = self.fit_model(warmup_data)
        ar_model = AutoReg(warmup_data, lags=optimal_lag, old_names=False).fit()

        i = ar_model.resid
        mean_e = np.mean(i)
        std_e = np.std(i)

        self.AR_params = {"model": ar_model, "lag": optimal_lag}
        self.f0 = norm(mean_e, std_e)
        self.rejection_count = 0

    #Algorithm to detect CP
    def detect_change_point(self, data, s):

        cpd_detected = False

        for i in range(self.n0 * s, len(data), s):
            e, cdf_values = self.predict_and_test(data[:i], s)

            # Using the 5-sigma rule
            rejected = np.sum(cdf_values < 0.05) > self.M

            if rejected:
                cpd_detected = True
                break

        return cpd_detected

    #Predict the next window using the AR model and perform hypothesis testing
    def predict_and_test(self, data, window_size):

        ar_model = self.AR_params["model"]
        pred = ar_model.predict(start=len(data), end=len(data) + window_size - 1)
        e = data[-window_size:] - pred

        return e, self.f0.cdf(e)

In [ ]:
def detect_change_points(data, baseline_cpd, s):
    change_points = []
    step_size = s

    for i in range(baseline_cpd.n0 * s, len(data), step_size):
        cpd_detected = baseline_cpd.detect_change_point(data[:i], s)
        if cpd_detected:
            change_points.append(i)

    return change_points


In [ ]:
#Parameters: n0 - warmup period length, n1 - window size length, M - rejection threshold, K - cumulative divergence threshold
n0 = 10
n1 = 64
M = 6
K = 10
s = 100

baseline_cpd = BaselineCPDEarth(n0, M, K)


baseline_cpd.warmup(Test2, s)

#Prediction Phase: detect change points
predicted_bkps = detect_change_points(Test2, baseline_cpd, s)


print(f"Change points prediction: {predicted_bkps}")

Change points prediction: [1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600, 9700, 9800]


In [ ]:
#Parameters: n0 - warmup period length, n1 - window size length, M - rejection threshold, K - cumulative divergence threshold
n0 = 10
n1 = 64
M = 6
K = 10
s = 100

baseline_cpd = BaselineCPDEarth(n0, M, K)


baseline_cpd.warmup(Earth2, s)

#Prediction Phase: detect change points
predicted_bkps = detect_change_points(Earth2, baseline_cpd, s)


print(f"Change points prediction: {predicted_bkps}")

Change points prediction: [1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700]
